In [1]:
# general tools
import os
import re
import sys
import time
import h5py
import random
from glob import glob

import numpy as np
from datetime import datetime, timedelta
from random import shuffle

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend
from keras_unet_collection import utils as k_utils

sys.path.insert(0, '/glade/u/home/ksha/NCAR/')
sys.path.insert(0, '/glade/u/home/ksha/NCAR/libs/')

from namelist import *
import data_utils as du

# import argparse

# parser = argparse.ArgumentParser()
# parser.add_argument('lead1', help='lead1')
# parser.add_argument('lead2', help='lead2')
# parser.add_argument('lead3', help='lead3')
# parser.add_argument('lead4', help='lead4')

# parser.add_argument('lead_name', help='lead_name')
# parser.add_argument('model_tag', help='model_tag')

# args = vars(parser.parse_args())


2023-03-07 11:38:55.745837: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
lead1 = 2
lead2 = 3
lead3 = 4
lead4 = 5

lead_name = 4

In [3]:
def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def verif_metric(VALID_target, Y_pred, ref):
    BS = np.mean((VALID_target.ravel() - Y_pred.ravel())**2)
    metric = BS
    return metric / ref

In [4]:
filepath_vec = "/glade/work/ksha/NCAR/"

path_name1_v3 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v3/'
path_name2_v3 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v3/' 
path_name3_v3 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v3/'
path_name4_v3 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v3/'

path_name1_v4 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4x/'
path_name2_v4 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4x/'
path_name3_v4 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4x/'
path_name4_v4 = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4x/'

path_name1_v4_test = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4_temp/'
path_name2_v4_test = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4_temp/'
path_name3_v4_test = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4_temp/'
path_name4_v4_test = '/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4_temp/'

In [5]:
filename_train_lead1 = sorted(glob("{}TRAIN*lead{}.npy".format(path_name1_v3, lead1)))
filename_train_lead2 = sorted(glob("{}TRAIN*lead{}.npy".format(path_name1_v3, lead2)))
filename_train_lead3 = sorted(glob("{}TRAIN*lead{}.npy".format(path_name1_v3, lead3)))
filename_train_lead4 = sorted(glob("{}TRAIN*lead{}.npy".format(path_name1_v3, lead4)))

In [6]:
filename_valid_lead1 = sorted(glob("{}VALID*lead{}.npy".format(path_name1_v3, lead1)))
filename_valid_lead2 = sorted(glob("{}VALID*lead{}.npy".format(path_name1_v3, lead2)))
filename_valid_lead3 = sorted(glob("{}VALID*lead{}.npy".format(path_name1_v3, lead3)))
filename_valid_lead4 = sorted(glob("{}VALID*lead{}.npy".format(path_name1_v3, lead4)))

In [7]:
filename_test_lead1 = sorted(glob("{}TEST*lead{}.npy".format(path_name1_v4_test, lead1)))
filename_test_lead2 = sorted(glob("{}TEST*lead{}.npy".format(path_name1_v4_test, lead2)))
filename_test_lead3 = sorted(glob("{}TEST*lead{}.npy".format(path_name1_v4_test, lead3)))
filename_test_lead4 = sorted(glob("{}TEST*lead{}.npy".format(path_name1_v4_test, lead4)))

In [8]:
data_lead1 = np.load('/glade/work/ksha/NCAR/TRAIN_MINMAX_lead{}.npy'.format(lead1), allow_pickle=True)
data_lead2 = np.load('/glade/work/ksha/NCAR/TRAIN_MINMAX_lead{}.npy'.format(lead2), allow_pickle=True)
data_lead3 = np.load('/glade/work/ksha/NCAR/TRAIN_MINMAX_lead{}.npy'.format(lead3), allow_pickle=True)
data_lead4 = np.load('/glade/work/ksha/NCAR/TRAIN_MINMAX_lead{}.npy'.format(lead4), allow_pickle=True)

In [23]:
TRAIN_MAX_lead1 = data_lead1[()]['TRAIN_MAX']
TRAIN_MEAN_lead1 = data_lead1[()]['TRAIN_MEAN']

TRAIN_MAX_lead2 = data_lead2[()]['TRAIN_MAX']
TRAIN_MEAN_lead2 = data_lead2[()]['TRAIN_MEAN']

TRAIN_MAX_lead3 = data_lead3[()]['TRAIN_MAX']
TRAIN_MEAN_lead3 = data_lead3[()]['TRAIN_MEAN']

TRAIN_MAX_lead4 = data_lead4[()]['TRAIN_MAX']
TRAIN_MEAN_lead4 = data_lead4[()]['TRAIN_MEAN']

TEST_MAX_lead1 = data_lead1[()]['TEST_MAX']
TEST_MEAN_lead1 = data_lead1[()]['TEST_MEAN']

TEST_MAX_lead2 = data_lead2[()]['TEST_MAX']
TEST_MEAN_lead2 = data_lead2[()]['TEST_MEAN']

TEST_MAX_lead3 = data_lead3[()]['TEST_MAX']
TEST_MEAN_lead3 = data_lead3[()]['TEST_MEAN']

TEST_MAX_lead4 = data_lead4[()]['TEST_MAX']
TEST_MEAN_lead4 = data_lead4[()]['TEST_MEAN']

In [24]:
IND_TRAIN_lead = np.load('/glade/work/ksha/NCAR/IND_TRAIN_lead_full.npy', allow_pickle=True)[()]
TRAIN_ind1_v3 = IND_TRAIN_lead['lead{}'.format(lead1)]
TRAIN_ind2_v3 = IND_TRAIN_lead['lead{}'.format(lead2)]
TRAIN_ind3_v3 = IND_TRAIN_lead['lead{}'.format(lead3)]
TRAIN_ind4_v3 = IND_TRAIN_lead['lead{}'.format(lead4)]

IND_VALID_lead = np.load('/glade/work/ksha/NCAR/IND_VALID_lead_full.npy', allow_pickle=True)[()]
VALID_ind1_v3 = IND_VALID_lead['lead{}'.format(lead1)]
VALID_ind2_v3 = IND_VALID_lead['lead{}'.format(lead2)]
VALID_ind3_v3 = IND_VALID_lead['lead{}'.format(lead3)]
VALID_ind4_v3 = IND_VALID_lead['lead{}'.format(lead4)]

IND_TEST_lead = np.load('/glade/work/ksha/NCAR/IND_TEST_lead_v4.npy', allow_pickle=True)[()]

TEST_ind1 = IND_TEST_lead['lead{}'.format(lead1)]
TEST_ind2 = IND_TEST_lead['lead{}'.format(lead2)]
TEST_ind3 = IND_TEST_lead['lead{}'.format(lead3)]
TEST_ind4 = IND_TEST_lead['lead{}'.format(lead4)]



In [26]:
L = len(TRAIN_ind2_v3)

filename_train1_pick_v3 = []
filename_train2_pick_v3 = []
filename_train3_pick_v3 = []
filename_train4_pick_v3 = []

TRAIN_X_lead1 = np.empty((L, 30))
TRAIN_X_lead2 = np.empty((L, 30))
TRAIN_X_lead3 = np.empty((L, 30))
TRAIN_X_lead4 = np.empty((L, 30))

TRAIN_Y_v3 = np.empty(L)

for i in range(L):
    
    ind_lead1_v3 = int(TRAIN_ind1_v3[i])
    ind_lead2_v3 = int(TRAIN_ind2_v3[i])
    ind_lead3_v3 = int(TRAIN_ind3_v3[i])
    ind_lead4_v3 = int(TRAIN_ind4_v3[i])
    
    filename_train1_pick_v3.append(filename_train_lead1[ind_lead1_v3])
    filename_train2_pick_v3.append(filename_train_lead2[ind_lead2_v3])
    filename_train3_pick_v3.append(filename_train_lead3[ind_lead3_v3])
    filename_train4_pick_v3.append(filename_train_lead4[ind_lead4_v3])
    
    TRAIN_X_lead1[i, 0:15] = TRAIN_MAX_lead1[ind_lead1_v3, :]
    TRAIN_X_lead1[i, 15:30] = TRAIN_MEAN_lead1[ind_lead1_v3, :]
    
    TRAIN_X_lead2[i, 0:15] = TRAIN_MAX_lead2[ind_lead2_v3, :]
    TRAIN_X_lead2[i, 15:30] = TRAIN_MEAN_lead2[ind_lead2_v3, :]
    
    TRAIN_X_lead3[i, 0:15] = TRAIN_MAX_lead3[ind_lead3_v3, :]
    TRAIN_X_lead3[i, 15:30] = TRAIN_MEAN_lead3[ind_lead3_v3, :]
    
    TRAIN_X_lead4[i, 0:15] = TRAIN_MAX_lead4[ind_lead4_v3, :]
    TRAIN_X_lead4[i, 15:30] = TRAIN_MEAN_lead4[ind_lead4_v3, :]
    
    #TRAIN_Y_v3[i] = TRAIN_lead3_y_v3[ind_lead3_v3]
    
    if 'pos' in filename_train_lead3[ind_lead3_v3]:
        TRAIN_Y_v3[i] = 1.0
    else:
        TRAIN_Y_v3[i] = 0.0

In [ ]:
#where valid starts
L_valid1 = len(filename_train_lead1)
L_valid2 = len(filename_train_lead2)
L_valid3 = len(filename_train_lead3)
L_valid4 = len(filename_train_lead4)

In [27]:
L = len(VALID_ind2_v3)

filename_valid1_pick_v3 = []
filename_valid2_pick_v3 = []
filename_valid3_pick_v3 = []
filename_valid4_pick_v3 = []

VALID_X_lead1 = np.empty((L, 30))
VALID_X_lead2 = np.empty((L, 30))
VALID_X_lead3 = np.empty((L, 30))
VALID_X_lead4 = np.empty((L, 30))

VALID_Y_v3 = np.empty(L)

for i in range(L):
    
    ind_lead1_v3 = int(VALID_ind1_v3[i])
    ind_lead2_v3 = int(VALID_ind2_v3[i])
    ind_lead3_v3 = int(VALID_ind3_v3[i])
    ind_lead4_v3 = int(VALID_ind4_v3[i])
    
    filename_valid1_pick_v3.append(filename_valid_lead1[ind_lead1_v3])
    filename_valid2_pick_v3.append(filename_valid_lead2[ind_lead2_v3])
    filename_valid3_pick_v3.append(filename_valid_lead3[ind_lead3_v3])
    filename_valid4_pick_v3.append(filename_valid_lead4[ind_lead4_v3])
    
    VALID_X_lead1[i, 0:15] = TRAIN_MAX_lead1[ind_lead1_v3+L_valid1, :]
    VALID_X_lead1[i, 15:30] = TRAIN_MEAN_lead1[ind_lead1_v3+L_valid1, :]
    
    VALID_X_lead2[i, 0:15] = TRAIN_MAX_lead2[ind_lead2_v3+L_valid2, :]
    VALID_X_lead2[i, 15:30] = TRAIN_MEAN_lead2[ind_lead2_v3+L_valid2, :]
    
    VALID_X_lead3[i, 0:15] = TRAIN_MAX_lead3[ind_lead3_v3+L_valid3, :]
    VALID_X_lead3[i, 15:30] = TRAIN_MEAN_lead3[ind_lead3_v3+L_valid3, :]
    
    VALID_X_lead4[i, 0:15] = TRAIN_MAX_lead4[ind_lead4_v3+L_valid4, :]
    VALID_X_lead4[i, 15:30] = TRAIN_MEAN_lead4[ind_lead4_v3+L_valid4, :]
    
    #TRAIN_Y_v3[i] = TRAIN_lead3_y_v3[ind_lead3_v3]
    
    if 'pos' in filename_valid_lead3[ind_lead3_v3]:
        VALID_Y_v3[i] = 1.0
    else:
        VALID_Y_v3[i] = 0.0

In [31]:
ALL_lead1 = np.concatenate((TRAIN_X_lead1, VALID_X_lead1), axis=0)
ALL_lead2 = np.concatenate((TRAIN_X_lead2, VALID_X_lead2), axis=0)
ALL_lead3 = np.concatenate((TRAIN_X_lead3, VALID_X_lead3), axis=0)
ALL_lead4 = np.concatenate((TRAIN_X_lead4, VALID_X_lead4), axis=0)

ALL_X = np.concatenate((ALL_lead1, ALL_lead2, ALL_lead3, ALL_lead4), axis=1)
ALL_Y = np.concatenate((TRAIN_Y_v3, VALID_Y_v3), axis=0)

In [32]:
TRAIN_pos_v3 = ALL_X[ALL_Y==1, ...]
TRAIN_neg_v3 = ALL_X[ALL_Y==0, ...]

In [33]:
L = len(TEST_ind2)

filename_test1_pick_v3 = []
filename_test2_pick_v3 = []
filename_test3_pick_v3 = []
filename_test4_pick_v3 = []

TEST_X_lead1 = np.empty((L, 30))
TEST_X_lead2 = np.empty((L, 30))
TEST_X_lead3 = np.empty((L, 30))
TEST_X_lead4 = np.empty((L, 30))

TEST_Y_v3 = np.empty(L)

for i in range(L):
    
    ind_lead1_v3 = int(TEST_ind1[i])
    ind_lead2_v3 = int(TEST_ind2[i])
    ind_lead3_v3 = int(TEST_ind3[i])
    ind_lead4_v3 = int(TEST_ind4[i])
    
    filename_test1_pick_v3.append(filename_test_lead1[ind_lead1_v3])
    filename_test2_pick_v3.append(filename_test_lead2[ind_lead2_v3])
    filename_test3_pick_v3.append(filename_test_lead3[ind_lead3_v3])
    filename_test4_pick_v3.append(filename_test_lead4[ind_lead4_v3])
    
    TEST_X_lead1[i, 0:15] = TEST_MAX_lead1[ind_lead1_v3, :]
    TEST_X_lead1[i, 15:30] = TEST_MEAN_lead1[ind_lead1_v3, :]
    
    TEST_X_lead2[i, 0:15] = TEST_MAX_lead2[ind_lead2_v3, :]
    TEST_X_lead2[i, 15:30] = TEST_MEAN_lead2[ind_lead2_v3, :]
    
    TEST_X_lead3[i, 0:15] = TEST_MAX_lead3[ind_lead3_v3, :]
    TEST_X_lead3[i, 15:30] = TEST_MEAN_lead3[ind_lead3_v3, :]
    
    TEST_X_lead4[i, 0:15] = TEST_MAX_lead4[ind_lead4_v3, :]
    TEST_X_lead4[i, 15:30] = TEST_MEAN_lead4[ind_lead4_v3, :]
    
    #TRAIN_Y_v3[i] = TRAIN_lead3_y_v3[ind_lead3_v3]
    
    if 'pos' in filename_test_lead3[ind_lead3_v3]:
        TEST_Y_v3[i] = 1.0
    else:
        TEST_Y_v3[i] = 0.0

In [34]:
ALL_TEST_X = np.concatenate((TEST_X_lead1, TEST_X_lead2, TEST_X_lead3, TEST_X_lead4), axis=1)
ALL_TEST_Y = TEST_Y_v3

In [39]:
filename_test4_pick_v3[10]

'/glade/campaign/cisl/aiml/ksha/NCAR_batch_v4_temp/TESTv4_day029_neg_neg_neg_indx18_indy49_lead5.npy'

In [35]:
# def create_model():
    
#     IN = keras.Input((4, 30))
#     X = IN
#     X = keras.layers.Conv1D(128, kernel_size=2, strides=1, padding='valid')(X)
#     X = keras.layers.Activation("gelu")(X)
    
#     X = keras.layers.GlobalMaxPool1D()(X) #X = keras.layers.Flatten()(X)

#     X = keras.layers.Dense(64)(X)
#     X = keras.layers.Activation("relu")(X)
#     X = keras.layers.BatchNormalization()(X)

#     OUT = X
#     OUT = keras.layers.Dense(1, activation='sigmoid', bias_initializer=keras.initializers.Constant(-10))(OUT)

#     model = keras.models.Model(inputs=IN, outputs=OUT)
#     return model

def create_model():

    
    IN = keras.Input((120,))
    X = IN
    #
    X = keras.layers.Dense(64, activity_regularizer=keras.regularizers.L2(1e-2))(X)
    X = keras.layers.BatchNormalization()(X)
    X = keras.layers.Activation("relu")(X)
    
    OUT = X
    OUT = keras.layers.Dense(1, activation='sigmoid', bias_initializer=keras.initializers.Constant(-10))(OUT)

    model = keras.models.Model(inputs=IN, outputs=OUT)
    
    return model


In [36]:

seeds = [12342, 2536234, 98765, 473, 865, 7456, 69472, 3456357, 3425, 678,
         2452624, 5787, 235362, 67896, 98454, 12445, 46767, 78906, 345, 8695, 
         2463725, 4734, 23234, 884, 2341, 362, 5, 234, 483, 785356, 23425, 3621, 
         58461, 80968765, 123, 425633, 5646, 67635, 76785, 34214]

training_rounds = len(seeds)

ref = np.sum(ALL_TEST_Y) / len(ALL_TEST_Y)

# =========== Model Section ========== #

batch_dir = '/glade/scratch/ksha/DATA/NCAR_batch/'
temp_dir = '/glade/work/ksha/NCAR/Keras_models/'

key = '{}_lead{}'.format('test', 4)

model_name = '{}'.format(key)
model_path = temp_dir+model_name

tol = 0

# ========== Training loop ========== #
L_pos = len(TRAIN_pos_v3)
L_neg = len(TRAIN_neg_v3)

record = 1.1
print("Initial record: {}".format(record))

min_del = 0
max_tol = 100 # early stopping with patience

epochs = 500
batch_size = 64
L_train = 16

for r in range(training_rounds):
    if r == 0:
        tol = 0
    else:
        tol = -200

    model = create_model()
    #
    model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False),
                  optimizer=keras.optimizers.Adam(lr=1e-4))
    
    set_seeds(int(seeds[r]))
    print('Training round {}'.format(r))

    for i in range(epochs):            
        start_time = time.time()

        # loop of batch
        for j in range(L_train):
            N_pos = 32
            N_neg = batch_size - N_pos

            ind_neg = du.shuffle_ind(L_neg)
            ind_pos = du.shuffle_ind(L_pos)

            ind_neg_pick = ind_neg[:N_neg]
            ind_pos_pick = ind_pos[:N_pos]

            X_batch_neg = TRAIN_neg_v3[ind_neg_pick, :]
            X_batch_pos = TRAIN_pos_v3[ind_pos_pick, :]

            X_batch = np.concatenate((X_batch_neg, X_batch_pos), axis=0)

            Y_batch = np.ones([batch_size,])
            Y_batch[:N_neg] = 0.0

            ind_ = du.shuffle_ind(batch_size)

            X_batch = X_batch[ind_, :]
            Y_batch = Y_batch[ind_]

            # train on batch
            model.train_on_batch(X_batch, Y_batch);

        # epoch end operations
        Y_pred = model.predict([ALL_TEST_X,])

        Y_pred[Y_pred<0] = 0
        Y_pred[Y_pred>1] = 1

        record_temp = verif_metric(ALL_TEST_Y, Y_pred, ref)

        # if i % 10 == 0:
        #     model.save(model_path_backup)

        if (record - record_temp > min_del):
            print('Validation loss improved from {} to {}'.format(record, record_temp))
            record = record_temp
            tol = 0
            
            #print('tol: {}'.format(tol))
            # save
            print('save to: {}'.format(model_path))
            model.save(model_path)
        else:
            print('Validation loss {} NOT improved'.format(record_temp))
            if record_temp > 1.01:
                print('Early stopping')
                break;
            else:
                tol += 1
                if tol >= max_tol:
                    print('Early stopping')
                    break;
                else:
                    continue;
        print("--- %s seconds ---" % (time.time() - start_time))



Initial record: 1.1
Training round 0
Validation loss improved from 1.1 to 0.9998896972320412
save to: /glade/work/ksha/NCAR/Keras_models/test_lead4
INFO:tensorflow:Assets written to: /glade/work/ksha/NCAR/Keras_models/test_lead4/assets
--- 5.178749322891235 seconds ---
Validation loss 1.0007726997289714 NOT improved
Validation loss 1.0036550397416941 NOT improved
Validation loss 1.0073285917394195 NOT improved
Validation loss 1.013033279568838 NOT improved
Early stopping
Training round 1
Validation loss improved from 0.9998896972320412 to 0.9928250850136829
save to: /glade/work/ksha/NCAR/Keras_models/test_lead4
INFO:tensorflow:Assets written to: /glade/work/ksha/NCAR/Keras_models/test_lead4/assets
--- 4.9707653522491455 seconds ---
Validation loss 0.9941025732125967 NOT improved
Validation loss 0.9950920151546303 NOT improved
Validation loss 0.9964146942967965 NOT improved
Validation loss 0.9989560517935429 NOT improved
Validation loss 1.0050502482948152 NOT improved
Validation loss 1.

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x2b706c314b80>
Traceback (most recent call last):
  File "/glade/work/ksha/anaconda3/lib/python3.9/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 

KeyboardInterrupt

